In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from fpdf import FPDF
from PIL import Image

# Define file paths
norms_file_path = '/content/Faculty norm test.xlsx'
actuals_file_path = '/content/faculty intake norms.xlsx'
output_directory = '/faculty_comparison_charts'
pdf_output_path = '/content/Faculty_Comparison_Report.pdf'

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Load Excel files into pandas DataFrames
try:
    norms_df = pd.read_excel(norms_file_path)
    actuals_df = pd.read_excel(actuals_file_path)
except Exception as e:
    print(f"Error loading Excel files: {e}")

# Verify that both DataFrames have the same structure
if not norms_df.columns.equals(actuals_df.columns):
    raise ValueError("The columns in the two Excel files do not match. Please ensure they have the same structure.")

# Identify department column and role columns
department_column = norms_df.columns[0]
role_columns = norms_df.columns[1:]

# Initialize the PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# Iterate over each department and generate comparison bar charts
for department in norms_df[department_column].unique():
    # Extract norms and actuals for the current department
    norms_data = norms_df[norms_df[department_column] == department][role_columns]
    actuals_data = actuals_df[actuals_df[department_column] == department][role_columns]

    # Check if data exists for the department in both datasets
    if norms_data.empty or actuals_data.empty:
        print(f"Data for department '{department}' is missing in one of the files. Skipping this department.")
        continue

    # Flatten data to 1D arrays
    norms_values = norms_data.values.flatten()
    actuals_values = actuals_data.values.flatten()

    # Setup bar positions
    bar_width = 0.35
    indices = range(len(role_columns))

    # Create a new figure
    plt.figure(figsize=(12, 8))

    # Plot norms and actuals side by side
    plt.bar(indices, norms_values, width=bar_width, label='Norms', color='#1f77b4')
    plt.bar([i + bar_width for i in indices], actuals_values, width=bar_width, label='Actual Intake', color='#ff7f0e')

    # Set chart title and labels
    plt.xlabel('Faculty Roles', fontsize=12)
    plt.ylabel('Number of Faculty', fontsize=12)
    plt.title(f'Faculty Intake Comparison for {department}', fontsize=16)

    # Set x-ticks and labels
    plt.xticks([i + bar_width / 2 for i in indices], role_columns, rotation=45)

    # Add legend
    plt.legend()

    # Add grid for better readability
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Adjust layout to prevent clipping of labels
    plt.tight_layout()

    # Save the plot as a PNG file
    output_image_path = os.path.join(output_directory, f'{department}_faculty_comparison.png')
    plt.savefig(output_image_path, dpi=300)

    # Close the figure to free memory
    plt.close()

    # Add the saved image to the PDF
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt=f'Faculty Intake Comparison for {department}', ln=True, align='C')
    pdf.image(output_image_path, x=10, y=10, w=190)

    # Generate summary of underperforming roles
    summary_lines = []
    for role, norm_value, actual_value in zip(role_columns, norms_values, actuals_values):
        if actual_value < norm_value:
            summary_lines.append(f"- {role}: Actual intake is {actual_value}, which is less than the norm of {norm_value}.")

    if summary_lines:
        summary_text = "Summary :\n" + "\n".join(summary_lines)
    else:
        summary_text = "All roles meet or exceed the specified norms."

    # Add the summary to the PDF
    pdf.set_xy(10, 140)
    pdf.multi_cell(0, 10, txt=summary_text)

    print(f"Chart and summary saved and added to PDF for department: {department}")

# Save the final PDF
pdf.output(pdf_output_path)
print(f"Comparison PDF saved at: {pdf_output_path}")


<ipython-input-15-0e4366430c81>:87: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=12)
<ipython-input-15-0e4366430c81>:88: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt=f'Faculty Intake Comparison for {department}', ln=True, align='C')
<ipython-input-15-0e4366430c81>:88: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt=f'Faculty Intake Comparison for {department}', ln=True, align='C')
<ipython-input-15-0e4366430c81>:104: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.multi_cell(0, 10, txt=summary_text)


Chart and summary saved and added to PDF for department: Computer Engineering
Chart and summary saved and added to PDF for department: E&TC
Chart and summary saved and added to PDF for department: IT
Chart and summary saved and added to PDF for department: AIDS
Chart and summary saved and added to PDF for department: ECE
Chart and summary saved and added to PDF for department: FY
Chart and summary saved and added to PDF for department: CS(PG)
Chart and summary saved and added to PDF for department: DS(PG)
Comparison PDF saved at: /content/Faculty_Comparison_Report.pdf
